# StromGedachtApiMqttConnector - Sandbox
https://www.stromgedacht.de/

https://www.stromgedacht.de/api-info/

https://api.stromgedacht.de/index.html

https://www.stromgedacht.de/api-docs


###  Meaning of states (cf. https://www.stromgedacht.de):
- 1 = green\
- 2 = yellow\
- 3 = orange\
- 4 = red\
- 
#### Die Bedeutung der StromGedacht-Zustände ist – wie aus der StromGedacht-App bekannt – wie folgt:

- 1 = grüner Zustand: Normalbetrieb – Du musst nichts weiter tun\
- 2 = gelber Zustand: Verbrauch vorverlegen – Strom jetzt nutzen\
- 3 = oranger Zustand: Verbrauch reduzieren, um Kosten und CO2 zu sparen\
- 4 = roter Zustand: Verbrauch reduzieren, um Strommangel zu verhindern\

In [ ]:
import requests
import json
import pprint
import logging
from datetime import datetime, timedelta
from lib.stromgedacht_api import getDataNow, getDataStates
from lib.app_functions import convert2json, convert_to_mqtt_msg, publish2mqtt
from lib.basic_functions import read_config

In [ ]:
config = read_config('config.ini')
mqtt_topic = config.get('mqtt', 'topic')
stromgedachtURL = config.get('StromGedacht', 'url')
zip_code = config.get('StromGedacht', 'zip_code')

In [ ]:
# StromGedacht API
#
# Get current state for zip=70173 (Stuttgart)
response=getDataNow(stromgedachtURL,zip_code)
#pprint.pprint(response.status_code)
# content = json.loads(response.text)
# state = content["state"]
# print(state)
json_string = convert2json(response)
pprint.pprint(json_string)

MQTT_msg=convert_to_mqtt_msg(json_string,mqtt_topic)
publish2mqtt(MQTT_msg,config)


## get States within defined time range

In [ ]:
default_log_level = logging.DEBUG

In [ ]:
from lib.influx import *
influxdb_client = initinfluxDBclient(config)
check_db_status(influxdb_client, config.get('influxdb', 'database'))

In [ ]:
from lib.forecast import getDataStatesForecast, timestamp_convert
# Get states for zip=70173 (Stuttgart) from 2023-04-11T00:00:00+02:00 to 2023-04-17T23:59:59+02:00
# Please substitute the start and end dates above with valid dates as the API
# only provides data for up to four days in the past and two days in the future


response=getDataStatesForecast(stromgedachtURL,zip_code)
content = json.loads(response.text)
#pprint.pprint(response)
states = content["states"]
print(states)
#print(timestamp_convert(states[0]["from"]))
#print(timestamp_convert(states[0]["to"]))
#print(states[0]["state"])
#pprint.pprint(states[0])


In [ ]:
from lib.forecast import convert_dataset2influx
data = convert_dataset2influx(response,config)
write2influx(data,influxdb_client,config)
data